# SR Merged Product 메타데이터 생성

이 노트북은 `sr_merged_product` 테이블에 대한 통계 정보와 설명을 생성합니다.

## 작업 내용:
1. PostgreSQL 연결 및 테이블 스키마 확인
2. 테이블 통계 정보 수집
3. 컬럼별 통계 분석
4. Azure OpenAI를 활용한 설명 생성
5. 메타데이터 저장

In [15]:
# 1. 필요한 라이브러리 임포트
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# .env 파일 로드
load_dotenv()

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [16]:
# 2. PostgreSQL 연결 설정
def get_db_connection():
    """PostgreSQL 데이터베이스 연결"""
    try:
        conn = psycopg2.connect(
            host=os.getenv('PG_HOST'),
            port=os.getenv('PG_PORT'),
            database=os.getenv('PG_DATABASE'),
            user=os.getenv('PG_USER'),
            password=os.getenv('PG_PASSWORD')
        )
        print(f"✅ PostgreSQL 연결 성공: {os.getenv('PG_HOST')}")
        return conn
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# 연결 테스트
conn = get_db_connection()
if conn:
    conn.close()

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com


In [17]:
# 3. Azure OpenAI 클라이언트 설정
def get_openai_client():
    """Azure OpenAI 클라이언트 생성"""
    try:
        client = AzureOpenAI(
            azure_endpoint=os.getenv('ENDPOINT_URL'),
            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
            api_version="2024-12-01-preview"
        )
        print(f"✅ Azure OpenAI 클라이언트 생성 성공")
        return client
    except Exception as e:
        print(f"❌ Azure OpenAI 클라이언트 생성 실패: {e}")
        return None

# 클라이언트 테스트
openai_client = get_openai_client()

✅ Azure OpenAI 클라이언트 생성 성공


In [18]:
# 4. 테이블 스키마 정보 조회
def get_table_schema(table_name='sr_merged_product'):
    """테이블 스키마 정보 조회"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        query = """
        SELECT 
            column_name,
            data_type,
            character_maximum_length,
            numeric_precision,
            numeric_scale,
            is_nullable,
            column_default
        FROM information_schema.columns
        WHERE table_schema = 'public' 
        AND table_name = %s
        ORDER BY ordinal_position;
        """
        
        df_schema = pd.read_sql_query(query, conn, params=(table_name,))
        print(f"✅ 테이블 '{table_name}' 스키마 조회 성공")
        print(f"   - 컬럼 수: {len(df_schema)}")
        return df_schema
    
    except Exception as e:
        print(f"❌ 테이블 스키마 조회 실패: {e}")
        return None
    
    finally:
        conn.close()

# 스키마 조회
df_schema = get_table_schema()
if df_schema is not None:
    print("\n테이블 스키마 정보:")
    display(df_schema.head(10))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 테이블 'sr_merged_product' 스키마 조회 성공
   - 컬럼 수: 38

테이블 스키마 정보:


,column_name,data_type,character_maximum_length,numeric_precision,numeric_scale,is_nullable,column_default
0,disp_lv1,character varying,1000.0,NaN,NaN,YES,None
1,disp_lv2,character varying,1000.0,NaN,NaN,YES,None
2,disp_lv3,character varying,1000.0,NaN,NaN,YES,None
3,product_category_lv1,character varying,1000.0,NaN,NaN,YES,None
4,product_category_lv2,character varying,1000.0,NaN,NaN,YES,None
5,product_category_lv3,character varying,1000.0,NaN,NaN,YES,None
6,model_name,character varying,1000.0,NaN,NaN,YES,None
7,mdl_code,character varying,1000.0,NaN,NaN,YES,None
8,goods_id,character varying,1000.0,NaN,NaN,YES,None
9,goods_nm,character varying,1000.0,NaN,NaN,YES,None


In [19]:
# 5. 테이블 기본 통계 정보 수집
def get_table_statistics(table_name='sr_merged_product'):
    """테이블 기본 통계 정보 수집"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        statistics = {}
        cursor = conn.cursor()
        
        # 테이블 행 수
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        statistics['total_rows'] = cursor.fetchone()[0]
        
        # 테이블 크기
        cursor.execute(f"""
            SELECT pg_size_pretty(pg_relation_size('{table_name}')) as table_size,
                   pg_size_pretty(pg_total_relation_size('{table_name}')) as total_size
        """)
        size_info = cursor.fetchone()
        if size_info:
            statistics['table_size'] = size_info[0]
            statistics['total_size'] = size_info[1]
        
        # 테이블 통계 정보 (pg_stat_user_tables의 실제 컬럼명 사용)
        cursor.execute(f"""
            SELECT schemaname, relname, n_tup_ins, n_tup_upd, n_tup_del,
                   last_vacuum, last_autovacuum, last_analyze, last_autoanalyze
            FROM pg_stat_user_tables
            WHERE relname = '{table_name}'
        """)
        stat_info = cursor.fetchone()
        if stat_info:
            statistics['stats'] = {
                'schema': stat_info[0],
                'table': stat_info[1],
                'inserts': stat_info[2],
                'updates': stat_info[3],
                'deletes': stat_info[4],
                'last_vacuum': str(stat_info[5]) if stat_info[5] else None,
                'last_autovacuum': str(stat_info[6]) if stat_info[6] else None,
                'last_analyze': str(stat_info[7]) if stat_info[7] else None,
                'last_autoanalyze': str(stat_info[8]) if stat_info[8] else None
            }
        
        print(f"✅ 테이블 '{table_name}' 통계 정보 수집 완료")
        return statistics
    
    except Exception as e:
        print(f"❌ 테이블 통계 정보 수집 실패: {e}")
        # 에러 발생 시에도 기본 정보는 반환
        try:
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            return {'total_rows': count}
        except:
            return None
    
    finally:
        if 'cursor' in locals():
            cursor.close()
        conn.close()

# 통계 정보 수집
table_stats = get_table_statistics()
if table_stats:
    print("\n테이블 통계 정보:")
    print(json.dumps(table_stats, indent=2, default=str))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 테이블 'sr_merged_product' 통계 정보 수집 완료

테이블 통계 정보:
{
  "total_rows": 3492,
  "table_size": "3328 kB",
  "total_size": "11 MB",
  "stats": {
    "schema": "public",
    "table": "sr_merged_product",
    "inserts": 3508,
    "updates": 0,
    "deletes": 16,
    "last_vacuum": null,
    "last_autovacuum": "2025-09-23 14:48:07.251185+00:00",
    "last_analyze": null,
    "last_autoanalyze": "2025-09-23 14:48:12.692470+00:00"
  }
}


In [21]:
# 6. 컬럼별 상세 통계 분석
def get_column_statistics(table_name='sr_merged_product', sample_size=10000):
    """컬럼별 상세 통계 정보 수집"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # 먼저 테이블에 데이터가 있는지 확인
        cursor = conn.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        total_rows = cursor.fetchone()[0]
        
        if total_rows == 0:
            print(f"⚠️ 테이블 '{table_name}'에 데이터가 없습니다.")
            return None
        
        # 샘플 크기 조정 (전체 행 수보다 크면 전체 행 수로 조정)
        actual_sample_size = min(sample_size, total_rows)
        
        # 샘플 데이터 로드
        query = f"SELECT * FROM {table_name} LIMIT {actual_sample_size}"
        df = pd.read_sql_query(query, conn)
        
        column_stats = []
        
        for col in df.columns:
            stats = {
                'column_name': col,
                'data_type': str(df[col].dtype),
                'non_null_count': int(df[col].notna().sum()),
                'null_count': int(df[col].isna().sum()),
                'null_ratio': f"{df[col].isna().mean() * 100:.2f}%"
            }
            
            # unique_count 계산 시 에러 처리
            try:
                # JSON/JSONB 컬럼이나 복잡한 객체가 포함된 경우를 처리
                if df[col].dtype == 'object':
                    # 문자열로 변환하여 고유값 계산
                    unique_count = df[col].astype(str).nunique()
                else:
                    unique_count = df[col].nunique()
                
                stats['unique_count'] = int(unique_count)
                stats['unique_ratio'] = f"{unique_count / len(df) * 100:.2f}%"
            except Exception as e:
                # 에러 발생 시 기본값 설정
                stats['unique_count'] = -1  # -1은 계산 불가를 의미
                stats['unique_ratio'] = "N/A"
                print(f"  ⚠️ 컬럼 '{col}' unique_count 계산 실패: {e}")
            
            # 수치형 데이터 통계
            if pd.api.types.is_numeric_dtype(df[col]):
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:
                    stats.update({
                        'min': float(non_null_values.min()) if not pd.isna(non_null_values.min()) else None,
                        'max': float(non_null_values.max()) if not pd.isna(non_null_values.max()) else None,
                        'mean': float(non_null_values.mean()) if not pd.isna(non_null_values.mean()) else None,
                        'median': float(non_null_values.median()) if not pd.isna(non_null_values.median()) else None,
                        'std': float(non_null_values.std()) if not pd.isna(non_null_values.std()) else None
                    })
            
            # 문자열 및 객체 데이터 통계
            elif df[col].dtype == 'object':
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:
                    try:
                        # 문자열로 변환하여 길이 계산
                        str_values = non_null_values.astype(str)
                        str_lengths = str_values.str.len()
                        
                        stats.update({
                            'min_length': int(str_lengths.min()) if len(str_lengths) > 0 else 0,
                            'max_length': int(str_lengths.max()) if len(str_lengths) > 0 else 0,
                            'avg_length': float(str_lengths.mean()) if len(str_lengths) > 0 else 0
                        })
                        
                        # most_common 계산 시 에러 처리
                        try:
                            # 복잡한 객체는 문자열로 변환하여 카운트
                            value_counts = df[col].astype(str).value_counts().head(3)
                            stats['most_common'] = value_counts.to_dict()
                        except:
                            stats['most_common'] = {}
                            
                    except Exception as e:
                        print(f"  ⚠️ 컬럼 '{col}' 문자열 통계 계산 실패: {e}")
                        stats.update({
                            'min_length': -1,
                            'max_length': -1,
                            'avg_length': -1,
                            'most_common': {}
                        })
            
            # 날짜형 데이터 통계
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:
                    stats.update({
                        'min_date': str(non_null_values.min()),
                        'max_date': str(non_null_values.max()),
                        'date_range': str(non_null_values.max() - non_null_values.min())
                    })
            
            column_stats.append(stats)
        
        print(f"✅ 컬럼별 통계 분석 완료 (샘플 크기: {len(df)}행)")
        return pd.DataFrame(column_stats)
    
    except Exception as e:
        print(f"❌ 컬럼별 통계 분석 실패: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    finally:
        if 'cursor' in locals():
            cursor.close()
        conn.close()

# 컬럼 통계 수집
df_column_stats = get_column_statistics()
if df_column_stats is not None:
    print(f"\n총 {len(df_column_stats)}개 컬럼 분석 완료")
    print("\n컬럼별 통계 정보 (처음 10개):")
    display(df_column_stats.head(10))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 컬럼별 통계 분석 완료 (샘플 크기: 3492행)

총 38개 컬럼 분석 완료

컬럼별 통계 정보 (처음 10개):


,column_name,data_type,non_null_count,null_count,null_ratio,unique_count,unique_ratio,min_length,max_length,avg_length,most_common,min,max,mean,median,std
0,disp_lv1,object,3492,0,0.00%,9,0.26%,2.0,7.0,3.949599,"{'모바일': 803, '리빙가전': 757, 'TV': 507}",NaN,NaN,NaN,NaN,NaN
1,disp_lv2,object,3492,0,0.00%,39,1.12%,2.0,18.0,4.350229,"{'에어컨': 851, 'TV': 841, '갤럭시 액세서리': 444}",NaN,NaN,NaN,NaN,NaN
2,disp_lv3,object,3492,0,0.00%,148,4.24%,2.0,30.0,8.605097,"{'케이스': 304, '홈멀티 에어컨': 264, '스탠드 에어컨': 259}",NaN,NaN,NaN,NaN,NaN
3,product_category_lv1,object,3492,0,0.00%,18,0.52%,2.0,22.0,12.338202,"{'Flat Panel Displayer': 969, 'AIR CONDITIONER...",NaN,NaN,NaN,NaN,NaN
4,product_category_lv2,object,3492,0,0.00%,56,1.60%,2.0,32.0,10.180412,"{'FAC': 809, 'QLED TV': 632, 'MOBILE-VPS PRODU...",NaN,NaN,NaN,NaN,NaN
5,product_category_lv3,object,3492,0,0.00%,117,3.35%,2.0,29.0,14.840779,"{'Bespoke Floor Air Conditioner': 701, 'Case':...",NaN,NaN,NaN,NaN,NaN
6,model_name,object,3480,12,0.34%,2113,60.51%,1.0,14.0,11.248276,"{'SM-S938N': 51, '-': 38, 'SM-S936N': 36}",NaN,NaN,NaN,NaN,NaN
7,mdl_code,object,3492,0,0.00%,2507,71.79%,7.0,18.0,12.916094,"{'SM-X626NZAEKOO': 2, 'AF80F18D24CRZL': 2, 'SM...",NaN,NaN,NaN,NaN,NaN
8,goods_id,object,3492,0,0.00%,3492,100.00%,10.0,10.0,10.000000,"{'G200246964': 1, 'G000433247': 1, 'G000405652...",NaN,NaN,NaN,NaN,NaN
9,goods_nm,object,3492,0,0.00%,1622,46.45%,7.0,120.0,35.910939,"{'Bespoke AI 무풍 클래식 56.9㎡ (선매립, 리모컨 포함)': 26, ...",NaN,NaN,NaN,NaN,NaN


In [22]:
# 7. Azure OpenAI를 활용한 컬럼 설명 생성
def generate_column_description(column_info, table_context='sr_merged_product'):
    """Azure OpenAI를 사용하여 컬럼 설명 생성"""
    
    if not openai_client:
        return None
    
    prompt = f"""
    다음 데이터베이스 컬럼에 대한 한국어 설명을 생성해주세요.
    
    테이블명: {table_context}
    컬럼 정보:
    - 컬럼명: {column_info.get('column_name')}
    - 데이터 타입: {column_info.get('data_type')}
    - NULL 비율: {column_info.get('null_ratio', 'N/A')}
    - 고유값 개수: {column_info.get('unique_count', 'N/A')}
    - 최소값: {column_info.get('min', 'N/A')}
    - 최대값: {column_info.get('max', 'N/A')}
    
    다음 형식으로 응답해주세요:
    1. 짧은 설명 (한 줄, 20자 이내)
    2. 상세 설명 (2-3줄, 비즈니스 의미 포함)
    3. 데이터 특성 (NULL 허용 여부, 값 범위 등)
    """
    
    try:
        response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "당신은 데이터베이스 전문가입니다. 컬럼의 비즈니스 의미를 명확하게 설명해주세요."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=300
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"❌ OpenAI 설명 생성 실패: {e}")
        return None

# 테스트: 첫 번째 컬럼에 대한 설명 생성
if df_column_stats is not None and len(df_column_stats) > 0:
    test_column = df_column_stats.iloc[0].to_dict()
    description = generate_column_description(test_column)
    print(f"컬럼 '{test_column['column_name']}' 설명:")
    print("-" * 50)
    print(description)

컬럼 'disp_lv1' 설명:
--------------------------------------------------
1. 짧은 설명: 1차 상품 분류 레벨  
2. 상세 설명: 이 컬럼은 제품의 1단계 분류 카테고리를 나타내며, 상품이 어떤 대분류에 속하는지 구분하는 데 사용됩니다. 이를 통해 상품군별 분석 및 카테고리별 매출 파악이 가능합니다.  
3. 데이터 특성: NULL 값을 허용하며, 총 9개의 고유값이 존재합니다. 값은 문자열(object) 타입으로, 일부 값은 결측치(NaN)입니다.


In [23]:
# 8. 모든 컬럼에 대한 메타데이터 생성 (배치 처리)
def generate_all_column_metadata(df_stats, batch_size=5):
    """모든 컬럼에 대한 메타데이터 생성"""
    
    if df_stats is None or len(df_stats) == 0:
        print("❌ 컬럼 통계 정보가 없습니다.")
        return None
    
    metadata_list = []
    total_columns = len(df_stats)
    
    print(f"총 {total_columns}개 컬럼에 대한 메타데이터 생성 시작...")
    
    for i in range(0, total_columns, batch_size):
        batch_end = min(i + batch_size, total_columns)
        print(f"\n배치 처리: {i+1}-{batch_end}/{total_columns}")
        
        for idx in range(i, batch_end):
            column_info = df_stats.iloc[idx].to_dict()
            column_name = column_info['column_name']
            
            print(f"  - {column_name} 처리 중...")
            
            # OpenAI 설명 생성
            description = generate_column_description(column_info)
            
            # 메타데이터 구성
            metadata = {
                'column_name': column_name,
                'data_type': column_info.get('data_type'),
                'null_ratio': column_info.get('null_ratio'),
                'unique_count': column_info.get('unique_count'),
                'description': description,
                'generated_at': datetime.now().isoformat()
            }
            
            # 수치형 데이터 추가 정보
            if 'min' in column_info:
                metadata.update({
                    'min': column_info.get('min'),
                    'max': column_info.get('max'),
                    'mean': column_info.get('mean'),
                    'median': column_info.get('median')
                })
            
            metadata_list.append(metadata)
            
            # API 호출 제한 방지를 위한 대기
            import time
            time.sleep(0.5)
    
    print(f"\n✅ 총 {len(metadata_list)}개 컬럼 메타데이터 생성 완료")
    return pd.DataFrame(metadata_list)

# 메타데이터 생성 (처음 10개 컬럼만)
df_metadata = generate_all_column_metadata(df_column_stats.head(10))
if df_metadata is not None:
    print("\n생성된 메타데이터:")
    display(df_metadata)

총 10개 컬럼에 대한 메타데이터 생성 시작...

배치 처리: 1-5/10
  - disp_lv1 처리 중...
  - disp_lv2 처리 중...
  - disp_lv3 처리 중...
  - product_category_lv1 처리 중...
  - product_category_lv2 처리 중...

배치 처리: 6-10/10
  - product_category_lv3 처리 중...
  - model_name 처리 중...
  - mdl_code 처리 중...
  - goods_id 처리 중...
  - goods_nm 처리 중...

✅ 총 10개 컬럼 메타데이터 생성 완료

생성된 메타데이터:


,column_name,data_type,null_ratio,unique_count,description,generated_at,min,max,mean,median
0,disp_lv1,object,0.00%,9,1. 짧은 설명: 1단계 카테고리 대분류 \n2. 상세 설명: disp_lv1 컬...,2025-09-24T23:56:36.630342,NaN,NaN,NaN,NaN
1,disp_lv2,object,0.00%,39,1. 짧은 설명: 2단계 카테고리 코드 \n2. 상세 설명: disp_lv2 컬럼...,2025-09-24T23:56:38.184884,NaN,NaN,NaN,NaN
2,disp_lv3,object,0.00%,148,1. 짧은 설명: 세 번째 디스플레이 레벨 카테고리 \n2. 상세 설명: 제품 또...,2025-09-24T23:56:39.625686,NaN,NaN,NaN,NaN
3,product_category_lv1,object,0.00%,18,1. 짧은 설명: 1차 제품 카테고리 \n2. 상세 설명: 제품의 주요 분류 단계...,2025-09-24T23:56:41.053873,NaN,NaN,NaN,NaN
4,product_category_lv2,object,0.00%,56,1. 짧은 설명: 2차 상품 카테고리 \n2. 상세 설명: 이 컬럼은 제품의 2단...,2025-09-24T23:56:42.588562,NaN,NaN,NaN,NaN
5,product_category_lv3,object,0.00%,117,1. 짧은 설명: 3단계 상품 카테고리\n\n2. 상세 설명: 이 컬럼은 상품의 세...,2025-09-24T23:56:44.067795,NaN,NaN,NaN,NaN
6,model_name,object,0.34%,2113,1. 짧은 설명: 제품 모델명 \n2. 상세 설명: 제품의 구체적인 모델 이름을 ...,2025-09-24T23:56:45.417533,NaN,NaN,NaN,NaN
7,mdl_code,object,0.00%,2507,1. 짧은 설명: 모델 코드 \n2. 상세 설명: 제품 또는 서비스의 모델을 식별...,2025-09-24T23:56:46.612904,NaN,NaN,NaN,NaN
8,goods_id,object,0.00%,3492,1. 짧은 설명: 상품 식별자\n\n2. 상세 설명: 상품 식별자인 goods_id...,2025-09-24T23:56:47.913534,NaN,NaN,NaN,NaN
9,goods_nm,object,0.00%,1622,1. 짧은 설명: 상품 이름 \n2. 상세 설명: 상품 이름 컬럼은 제품의 고유한...,2025-09-24T23:56:49.244287,NaN,NaN,NaN,NaN


In [26]:
# 9. 테이블 전체 설명 생성
def generate_table_description(table_name='sr_merged_product', table_stats=None, column_stats=None):
    """테이블 전체에 대한 종합적인 설명 생성"""
    
    if not openai_client:
        return None
    
    # 주요 컬럼 정보 추출
    key_columns = []
    if column_stats is not None and len(column_stats) > 0:
        key_columns = column_stats.head(10)['column_name'].tolist()
    
    prompt = f"""
    다음 데이터베이스 테이블에 대한 종합적인 설명을 한국어로 작성해주세요.
    
    테이블명: {table_name}
    
    테이블 통계:
    - 전체 행 수: {table_stats.get('total_rows', 'N/A') if table_stats else 'N/A'}
    - 테이블 크기: {table_stats.get('table_size', 'N/A') if table_stats else 'N/A'}
    - 전체 컬럼 수: {len(column_stats) if column_stats is not None else 'N/A'}
    
    주요 컬럼: {', '.join(key_columns)}
    
    다음 내용을 포함하여 설명해주세요:
    1. 테이블의 주요 목적과 역할 (2-3줄)
    2. 저장되는 데이터의 비즈니스 의미
    3. 다른 테이블과의 잠재적 연관 관계
    4. 데이터 활용 사례 (예: 리포트, 분석, API 등)
    """
    
    try:
        response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "당신은 데이터 아키텍트입니다. 테이블의 비즈니스 목적과 활용 방안을 명확하게 설명해주세요."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=500
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"❌ 테이블 설명 생성 실패: {e}")
        return None

# 테이블 설명 생성
table_description = generate_table_description(
    table_name='sr_merged_product',
    table_stats=table_stats,
    column_stats=df_column_stats
)

if table_description:
    print("=" * 60)
    print("📋 SR_MERGED_PRODUCT 테이블 설명")
    print("=" * 60)
    print(table_description)
    print("=" * 60)

📋 SR_MERGED_PRODUCT 테이블 설명
sr_merged_product 테이블은 다양한 유통 채널과 상품 정보를 통합하여 제품 카테고리, 모델명, 상품 ID 등 핵심 정보를 저장하는 데이터 저장소입니다. 이를 통해 제품별 분류와 분석을 지원하며, 상품 관리와 마케팅 전략 수립에 활용됩니다.

이 테이블에는 제품의 대분류, 중분류, 소분류(disp_lv1~disp_lv3, product_category_lv1~lv3), 모델명(model_name, mdl_code), 상품 ID(goods_id), 상품명(goods_nm) 등 상품의 세부 분류와 식별 정보를 담고 있어, 상품의 계층적 구조와 상세 정보를 파악할 수 있습니다. 이는 상품 데이터의 표준화와 통합 관리를 가능하게 하며, 다양한 유통 채널의 상품 정보를 하나의 표에서 통합적으로 조회할 수 있도록 합니다.

다른 테이블과의 잠재적 연관 관계로는, 판매 데이터, 재고 데이터, 가격 정보, 고객 데이터 등과 연계하여 상품별 판매 성과, 재고 상태, 가격 정책, 고객 선호도 분석 등에 활용될 수 있습니다. 예를 들어, 판매 실적 분석이나 재고 최적화, 마케팅 타겟팅 등에 중요한 기초 데이터로 사용됩니다.

이 테이블은 주로 내부 분석 리포트, 상품 카테고리별 보고서, 상품 성과 분석, API 기반 상품 정보 제공 등 다양한 비즈니스 활용 사례에 활용됩니다. 이를 통해 상품 관련 의사결정 지원과 고객 맞춤형 마케팅 전략 수립이 가능해집니다.


In [27]:
# 10. 메타데이터 저장 (JSON, CSV 형식)
def save_metadata(df_metadata, table_description, output_dir='./metadata'):
    """생성된 메타데이터를 파일로 저장"""
    
    import os
    from datetime import datetime
    
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. CSV 형식으로 컬럼 메타데이터 저장
    if df_metadata is not None:
        csv_path = os.path.join(output_dir, f'sr_merged_product_columns_{timestamp}.csv')
        df_metadata.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"✅ 컬럼 메타데이터 CSV 저장: {csv_path}")
    
    # 2. JSON 형식으로 전체 메타데이터 저장
    metadata_json = {
        'table_name': 'sr_merged_product',
        'generated_at': datetime.now().isoformat(),
        'table_description': table_description,
        'table_statistics': table_stats,
        'columns': df_metadata.to_dict('records') if df_metadata is not None else []
    }
    
    json_path = os.path.join(output_dir, f'sr_merged_product_metadata_{timestamp}.json')
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(metadata_json, f, ensure_ascii=False, indent=2, default=str)
    print(f"✅ 전체 메타데이터 JSON 저장: {json_path}")
    
    # 3. 테이블 설명만 별도 텍스트 파일로 저장
    if table_description:
        txt_path = os.path.join(output_dir, f'sr_merged_product_description_{timestamp}.txt')
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(f"SR_MERGED_PRODUCT 테이블 설명\n")
            f.write("=" * 60 + "\n")
            f.write(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 60 + "\n\n")
            f.write(table_description)
        print(f"✅ 테이블 설명 TXT 저장: {txt_path}")
    
    return {
        'csv_path': csv_path if df_metadata is not None else None,
        'json_path': json_path,
        'txt_path': txt_path if table_description else None
    }

# 메타데이터 저장
saved_files = save_metadata(df_metadata, table_description)
print("\n📁 저장된 파일:")
for key, path in saved_files.items():
    if path:
        print(f"  - {key}: {path}")

✅ 컬럼 메타데이터 CSV 저장: ./metadata/sr_merged_product_columns_20250924_235722.csv
✅ 전체 메타데이터 JSON 저장: ./metadata/sr_merged_product_metadata_20250924_235722.json
✅ 테이블 설명 TXT 저장: ./metadata/sr_merged_product_description_20250924_235722.txt

📁 저장된 파일:
  - csv_path: ./metadata/sr_merged_product_columns_20250924_235722.csv
  - json_path: ./metadata/sr_merged_product_metadata_20250924_235722.json
  - txt_path: ./metadata/sr_merged_product_description_20250924_235722.txt


In [28]:
# 11. 메타데이터 데이터베이스 저장 (선택사항)
def save_metadata_to_db(df_metadata, table_description, target_table='table_metadata'):
    """생성된 메타데이터를 데이터베이스에 저장"""
    
    conn = get_db_connection()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        
        # 메타데이터 테이블 생성 (없는 경우)
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {target_table} (
            id SERIAL PRIMARY KEY,
            table_name VARCHAR(255),
            column_name VARCHAR(255),
            data_type VARCHAR(100),
            null_ratio VARCHAR(20),
            unique_count INTEGER,
            min_value TEXT,
            max_value TEXT,
            mean_value NUMERIC,
            median_value NUMERIC,
            description TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_table_query)
        
        # 테이블 설명 저장용 테이블
        create_desc_table_query = """
        CREATE TABLE IF NOT EXISTS table_descriptions (
            id SERIAL PRIMARY KEY,
            table_name VARCHAR(255) UNIQUE,
            description TEXT,
            total_rows BIGINT,
            table_size VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_desc_table_query)
        
        # 기존 데이터 삭제 (중복 방지)
        cursor.execute(f"DELETE FROM {target_table} WHERE table_name = 'sr_merged_product'")
        
        # 컬럼 메타데이터 삽입
        if df_metadata is not None:
            for _, row in df_metadata.iterrows():
                insert_query = f"""
                INSERT INTO {target_table} 
                (table_name, column_name, data_type, null_ratio, unique_count, 
                 min_value, max_value, mean_value, median_value, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (
                    'sr_merged_product',
                    row.get('column_name'),
                    row.get('data_type'),
                    row.get('null_ratio'),
                    row.get('unique_count'),
                    str(row.get('min')) if 'min' in row else None,
                    str(row.get('max')) if 'max' in row else None,
                    row.get('mean') if 'mean' in row else None,
                    row.get('median') if 'median' in row else None,
                    row.get('description')
                ))
        
        # 테이블 설명 저장/업데이트
        if table_description and table_stats:
            upsert_query = """
            INSERT INTO table_descriptions (table_name, description, total_rows, table_size)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (table_name) 
            DO UPDATE SET 
                description = EXCLUDED.description,
                total_rows = EXCLUDED.total_rows,
                table_size = EXCLUDED.table_size,
                updated_at = CURRENT_TIMESTAMP
            """
            cursor.execute(upsert_query, (
                'sr_merged_product',
                table_description,
                table_stats.get('total_rows'),
                table_stats.get('table_size')
            ))
        
        conn.commit()
        print(f"✅ 메타데이터가 데이터베이스에 저장되었습니다.")
        return True
        
    except Exception as e:
        print(f"❌ 데이터베이스 저장 실패: {e}")
        conn.rollback()
        return False
        
    finally:
        cursor.close()
        conn.close()

# 데이터베이스에 저장
if df_metadata is not None:
    save_metadata_to_db(df_metadata, table_description)

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 메타데이터가 데이터베이스에 저장되었습니다.


## 요약

이 노트북은 `sr_merged_product` 테이블에 대한 메타데이터를 생성하고 저장합니다.

### 수행된 작업:
1. ✅ PostgreSQL 연결 설정
2. ✅ Azure OpenAI 클라이언트 구성
3. ✅ 테이블 스키마 정보 조회
4. ✅ 테이블 통계 정보 수집 (행 수, 크기 등)
5. ✅ 컬럼별 상세 통계 분석
6. ✅ Azure OpenAI를 활용한 컬럼 설명 생성
7. ✅ 테이블 전체 설명 생성
8. ✅ 메타데이터 파일 저장 (CSV, JSON, TXT)
9. ✅ 데이터베이스에 메타데이터 저장

### 다음 단계:
- 전체 컬럼에 대한 메타데이터 생성 (현재는 10개만 처리)
- 데이터 품질 검증 규칙 추가
- 비즈니스 용어 사전과 연계
- 다른 테이블에 대한 메타데이터 생성